In [227]:
# Fetching stock data from tiingo API
# retrieving data pogramatically using python
# python requests package: python library to make requests
# Response: JSON format 
# API Key: dfeeb56a6c0e72a4c31667b48bfb938fc21c8163
# Prediction using linear regression

In [228]:
import requests

In [229]:
# function for data request
def get_stock_data(stock_symbol, start_date, end_date):
    base_url = 'https://api.tiingo.com/tiingo/daily/' + stock_symbol + '/prices'
    api_key = 'dfeeb56a6c0e72a4c31667b48bfb938fc21c8163'
    payload = {
        'token': api_key,
        'startDate': start_date,
        'endDate': end_date
    }
    # requests library makes URL with base URL and payload on it's own
    response = requests.get(base_url, params=payload)
    return response 

In [230]:
# data_response = get_stock_data('AAPL', '2020-9-1', '2020-10-1')
# print(data_response.url)
# print(data_response.json())

In [231]:
# parsing the data
# function to get only date, low, high from response
def decode_data(response):
    json_response = response.json()
    dates = []
    highs = []
    lows = []
    closes = []
    opens = []
    vols = []

    for json_object in json_response:
        date = json_object['date']
        high = json_object['high']
        low = json_object['low']
        close = json_object['close']
        open = json_object['open']
        vol = json_object['volume']
        
        dates.append(date)
        highs.append(high)
        lows.append(low)
        closes.append(close)
        opens.append(open)
        vols.append(vol)
    
    return dates, highs, lows, closes, opens, vols

In [232]:
# calculate difference
def calculate_price_difference(opens, closes):
    price_differences = []

    for i in range(len(opens) - 1):
        if opens[i + 1] >= closes[i]:
            price_differences.append([1,0])

        else:
            price_differences.append([0,1])

    return price_differences

In [233]:
def create_datasets(stock_symbol, start_date, end_date):
    response = get_stock_data(stock_symbol, start_date, end_date)
    dates, highs, lows, closes, opens, vols = decode_data(response)

    print(response.url)
    labels = calculate_price_difference(opens, closes)

    inputs = []
    for i in range(len(labels)):
        inputs.append([closes[i], opens[i], highs[i], lows[i], vols[i]/1000])

    return inputs, labels


In [241]:
# 1 year train data
x_train, y_train = create_datasets('AMZN', '2019-1-1', '2020-2-1')
# one month test data
x_test, y_test = create_datasets('AMZN', '2020-1-1', '2020-2-1')

https://api.tiingo.com/tiingo/daily/AMZN/prices?token=dfeeb56a6c0e72a4c31667b48bfb938fc21c8163&startDate=2019-1-1&endDate=2020-2-1
https://api.tiingo.com/tiingo/daily/AMZN/prices?token=dfeeb56a6c0e72a4c31667b48bfb938fc21c8163&startDate=2020-1-1&endDate=2020-2-1


In [242]:
# creating model based on Linear regression
# [1,0] if the stock goes up
# [0,1] if the stock goes down
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [243]:
# y = Wx + b
x_input = tf.placeholder(dtype=tf.float32, shape=[None, 5])
y_input = tf.placeholder(dtype=tf.float32, shape=[None, 2])
W = tf.Variable(initial_value=tf.ones(shape=[5,2]))
b = tf.Variable(initial_value=tf.ones(shape=[2]))
y_output = tf.add(tf.matmul(x_input, W), b)

In [244]:
loss = tf.reduce_sum(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_input, logits=y_output)))
optimiser = tf.train.GradientDescentOptimizer(0.01)
train_step = optimiser.minimize(loss)

In [245]:
# Accuracy measurement
def measure_accuracy(actual, expected):
    num_correct = 0
    for i in range(len(actual)):
        actual_value = actual[i]
        expected_value = expected[i]
        if actual_value[0] >= actual_value[1] and expected_value[0] >= expected_value[1]:
            num_correct += 1
        elif actual_value[0] <= actual_value[1] and expected_value[0] <= expected_value[1]:
            num_correct += 1
        
    return (num_correct/len(actual))*100

In [246]:
# Runnning the model
session = tf.Session()
session.run(tf.global_variables_initializer())

In [247]:
for _ in range(30000):
    session.run(train_step, feed_dict={x_input: x_train,y_input: y_train})
print(measure_accuracy(session.run(y_output,feed_dict={x_input: x_test}),y_test))

70.0
